# LOKI Detector Diagnostics

This page is primarily intended for instrument scientists and other experts.

ESSsans has some tools for inspecting the LOKI detector based on recorded event data.
This notebook gives an overview of the available tools.

## Voxel Viewer

The [FlatDetectorViewer](../../generated/modules/ess.loki.diagnostics.FlatDetectorViewer.rst) is an interactive visualization for inspecting individual voxels.
It shows a 2D histogram of recorded events.
The image axes correspond to a chosen pair of logical dimensions combined with all other dimensions.
Each bin in the image corresponds to exactly one detector pixel.

In [ ]:
import scipp as sc
from ess.loki.diagnostics import FlatDetectorViewer

In [ ]:
%matplotlib widget

Load simulated test data and compute histograms:

In [ ]:
import scippnexus as sx

dg = sx.load("loki_999999_00009928.hdf")
dg

In [ ]:
data = {}
for i in range(9):
    key = f"loki_detector_{i}"
    data[key] = dg['entry']['instrument'][key]["detector_events"].fold(dim='detector_number', sizes={'layer': 4, 'tube': -1, 'straw': 7, 'pixel': 512}).hist()

data = sc.DataGroup(data)

In [ ]:
data['loki_detector_0']

Display 2D view:

In [ ]:
FlatDetectorViewer(data, norm='log', figsize=(9, 6))